In [20]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups_vectorized


In [4]:
data = fetch_20newsgroups_vectorized(remove=('headers', 'footers', 'quotes'))

In [5]:
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

In [6]:
X = data['data']

In [26]:
X.shape

(11314, 101631)

In [27]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=10, random_state=42)

In [29]:
U_times_S = svd.fit_transform(X)

In [30]:
S = svd.singular_values_

In [33]:
Vt = svd.components_

In [34]:
U_times_S.shape, S.shape, Vt.shape

((11314, 10), (10,), (10, 101631))

**Exercicio** Qual o significado de:

(a) Uma linha de `U_times_S`?

(b) Uma coluna de `Vt`

(c) Uma *coluna* de `U_times_S`?

(d) Uma *linha* de `Vt`?

(e) Os valores de `S`?